# 0. intro
1. https://www.bilibili.com/video/BV18g4119737?p=29&vd_source=70200f7d09862fd682e5f89b22c89125
2. 介绍了：
    - 基本四则运算
    - element-wise乘法。nn的降维/线性层常用
    - 还有

In [1]:
import torch

# 1. basic
1. 加减乘除,**一些写法是等效的**：
    - 加：+或者sum
    - 减：-或者sub
    - 乘：*或者mul
    - 除：/或者div
2. 有的会进行广播,比如：

In [2]:
a = torch.rand(3,4)
b = torch.rand(4)
print("a+b:\n",a + b)

a+b:
 tensor([[1.0658, 1.2585, 1.0679, 1.1287],
        [1.1720, 1.2874, 0.5183, 1.3710],
        [1.2377, 1.5517, 0.9622, 1.3792]])


3. 可以检查一下等效效果：

In [3]:
torch.all(torch.eq(a-b, torch.sub(a,b)))

tensor(True)

In [4]:
torch.all(torch.eq(a*b, torch.mul(a,b)))

tensor(True)

In [5]:
torch.all(torch.eq(a/b, torch.div(a,b)))

tensor(True)

# 2. element-wise乘法
1. `torch.mm`:
    - **element-wise**
    - only for 2d,所以不推荐
2. `torch.matmul` == `@`
    - **element-wise**
    - 推荐用这个,或者直接用@,是matmul的重载
    -

In [6]:
a = torch.tensor([[3.,3.],[3.,3.]])
b = torch.ones(2,2)

In [7]:
c_mm = torch.mm(a,b)
print("mm:",c_mm)

mm: tensor([[6., 6.],
        [6., 6.]])


In [8]:
c_matmul = torch.matmul(a,b)
print("matmul or @:",c_matmul)

matmul or @: tensor([[6., 6.],
        [6., 6.]])


3. example: 降维,nn中对线性层会用
    - 先用`view`函数打平成(4,784)的向量`x`,然后通过降维,得到一个(4,512)
    - 很显然“降维”通过矩阵乘法完成,`x`需要和(784,512)大小的矩阵`w`相乘
    - **注意pytorch的写法**：`w`的**第一个维度**是`chanel-out`。因此一般用`.t()`转置一下

In [9]:
x = torch.rand(4,784)
w = torch.rand(512,784)
# 降维
(x@w.t()).shape

torch.Size([4, 512])

4. 高维的mutmul
    - 我们希望的效果是对batch和channel这两个维度保持不变,对H\*W这个大小的矩阵进行一个 **“并行”** 的相乘
    -

In [11]:
a = torch.rand(4,3,28,64)
b = torch.rand(4,3,64,32)
ans = torch.matmul(a,b)
print("(28,64)*(64,32)得到的shape:",ans.shape)

(28,64)*(64,32)得到的shape: torch.Size([4, 3, 28, 32])


也可以使用broadcast机制,对`b`进行广播

In [ ]:
# broadcast
a = torch.rand(4,3,28,64)
b = torch.rand(4,1,64,32)
ans = torch.matmul(a,b)
ans.shape

# 3. 其他运算
1. `pow`和`**`的效果差不多
2. `.exp`和`log`互为反函数,注意后者是`e`为底,以2为底使用`log2`
## 3.1 近似
1. floor() ceil(). round()是四舍五入
2. trunc(),frac()分别裁为整数和小数部分

In [14]:
a = torch.tensor(3.14)
print("a.trunc():",a.trunc())
print("a.frac():",a.frac())

a.trunc(): tensor(3.)
a.frac(): tensor(0.1400)


3. 常用的是gradient clipping的操作：`clamp(threshold)`
    - 当训练时,梯度不稳定,一定要打印一下**梯度的模**：`w.grad.norm(2)`,**一般100或者1000都很大,小于10是比较合适的**
    - 所以要对梯度进行限制（不是对w进行限制！）

In [21]:
grad = torch.rand(2,3)*15
# print(f"{grad.max()},{grad.median()}")
print("grad:\n",grad)
print("从10截断")
print("grad.clamp(10):\n",grad.clamp(10))

grad:
 tensor([[2.8222, 7.4594, 5.4390],
        [7.9058, 7.4556, 3.6127]])
从10截断
grad.clamp(10):
 tensor([[10., 10., 10.],
        [10., 10., 10.]])
